<a href="https://colab.research.google.com/github/HarounH/smol/blob/main/rl/policy_gradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# @title Install dependencies
!pip install rarfile
!pip install ale-py
!pip install -q gymnasium
!pip install swig
!pip install pyvirtualdisplay
!pip install pyglet
!pip install pygame
!pip install minigrid
!pip install -q swig
!pip install minigrid
!pip3 install box2d-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.7/136.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.8-cp312-cp312-linux_x86_64.whl size=2399013 sha256=26b1a9804536aa32bab380836b6a084c94213a0f890c255aa572c7a354b66c48
  Stored in directory: /root/.cache/pip/wheels/d6/3c/ab/b6fd75459cadc56f4a4125d4cb387a708a59ca8589e4cc6b7d
Successfully built box2d-py


In [5]:
# @title Policy Gradients?

import gymnasium
import numpy as np
from collections import deque
from dataclasses import dataclass
import io
import base64
import optax

import numpy as np
import imageio.v2 as imageio
from IPython.display import HTML
import jax.numpy as jnp
from flax import struct
import jax

from IPython.display import HTML
import numpy as np
import imageio
import io, base64
from PIL import Image, ImageDraw, ImageFont
import time
from scipy.ndimage.filters import gaussian_filter1d
import orbax.checkpoint as ocp
from flax import nnx

def show_mp4_from_frames_imageio(
    frames,
    text=None,              # str or list[str]
    text_color=(255, 0, 0), # red
    text_padding=10,
    fps=30,
):
    """
    Display a list of (H, W, 3) RGB frames inline as MP4 with optional per-frame text.

    Args:
        frames: list of frames, each (H, W, 3), RGB.
        fps: frames per second.
        text: either a single string or list of strings (len == len(frames)).
        text_color: RGB tuple for text.
        text_padding: pixels offset from top-right corner.

    Returns:
        IPython.display.HTML object.
    """
    if not frames:
        raise ValueError("frames list is empty")

    # Normalize text argument
    if text is None:
        texts = [None] * len(frames)
    elif isinstance(text, str):
        texts = [text] * len(frames)
    else:
        if len(text) != len(frames):
            raise ValueError("text list must be same length as frames")
        texts = text

    norm_frames = []
    for idx, f in enumerate(frames):
        f = np.asarray(f)

        if f.ndim != 3 or f.shape[-1] != 3:
            raise ValueError(f"Frame must be (H,W,3), got {f.shape}")

        # Normalize dtype → uint8
        if f.dtype != np.uint8:
            if np.issubdtype(f.dtype, np.floating):
                f = np.clip(f, 0.0, 1.0)
                f = (f * 255).astype(np.uint8)
            else:
                f = np.clip(f, 0, 255).astype(np.uint8)

        # Convert to PIL for text overlay
        img = Image.fromarray(f)
        draw = ImageDraw.Draw(img)

        txt = texts[idx]
        if txt:
            # Choose a default font
            try:
                font = ImageFont.truetype("DejaVuSans.ttf", 20)
            except:
                font = ImageFont.load_default()

            # Width via textlength
            tw = draw.textlength(txt, font=font)
            # Height via font metrics
            ascent, descent = font.getmetrics()
            th = ascent + descent
            # Position: top-right
            H, W = f.shape[:2]
            x = W - tw - text_padding
            y = text_padding

            draw.text((x, y), txt, fill=text_color, font=font)

        norm_frames.append(np.asarray(img))

    # Encode MP4 into an in-memory buffer
    buf = io.BytesIO()
    imageio.mimsave(buf, norm_frames, format="mp4", fps=fps)
    buf.seek(0)
    video_bytes = buf.read()

    b64 = base64.b64encode(video_bytes).decode("ascii")

    html = f"""
    <video controls loop>
        <source src="data:video/mp4;base64,{b64}" type="video/mp4">
        Your browser does not support the video tag.
    </video>
    """
    return HTML(html)

/tmp/ipython-input-3061684467.py:24: DeprecationWarning: Please import `gaussian_filter1d` from the `scipy.ndimage` namespace; the `scipy.ndimage.filters` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.filters import gaussian_filter1d


In [6]:
env = gymnasium.make("LunarLander-v3", render_mode="rgb_array")
current_state, _ = env.reset()
print(env.action_space)
print(env.observation_space)

Discrete(4)
Box([ -2.5        -2.5       -10.        -10.         -6.2831855 -10.
  -0.         -0.       ], [ 2.5        2.5       10.        10.         6.2831855 10.
  1.         1.       ], (8,), float32)


In [8]:
@struct.dataclass
class Datum:
    s_t: jax.Array
    s_tp1: jax.Array
    r_t: jax.Array
    a_t: jax.Array
    done: bool
    on_policy_t: bool
    frame_hwc: np.ndarray | None = None

def play(
    policy_model,
    exploration_probability: float,
    env,
    current_state,
    rng,
    greedy: bool = True,
    render: bool = False,
) -> tuple[Datum, np.ndarray]:
    action = env.action_space.sample()
    on_policy_t = False
    if (policy_model is not None):
        if (jax.random.bernoulli(rng.params(), exploration_probability, shape=None)):
            pass
        else:
            pi_a = policy_model(current_state)
            if greedy:
                action = pi_a.argmax()
            else:
                action = jax.random.multinomial(rng.params(), 1, jnp.exp(pi_a))
            on_policy_t = True

    frame_hwc = env.render() if render else None
    new_state, reward, t0, t1, _ = env.step(action)
    done = (t0 or t1)
    datum = Datum(
        s_t=current_state,
        s_tp1=new_state,
        r_t=reward,
        a_t=action,
        done=done,
        on_policy_t=on_policy_t,
        frame_hwc=frame_hwc
    )
    return datum, new_state

n_steps = 100
trajectory = []
rngs = nnx.Rngs(default=1337)
state, _ = env.reset()
for step_idx in range(n_steps):
    datum, state = play(None, 1, env, state, rngs, True)
    trajectory.append(datum)


show_mp4_from_frames_imageio(
    [d.frame_hwc for d in trajectory],
    [f"action: {d.a_t}" for d in trajectory],
)

/usr/lib/python3.12/subprocess.py:1885: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _fork_exec(


In [ ]:
class Layer(nnx.Module):
    def __init__(self, dim: int, rngs: nnx.Rngs):
        self.fc1 = nnx.Linear(dim, dim, rngs=rngs)

    def __call__(self, x):
        z = self.fc1(x)
        z = jax.nn.relu(z)
        return z


class PolicyNetwork(nnx.Module):
    # same shape and stuff as QNet
    def __init__(self, dim_in: int = 4, d_model: int = 32, num_actions: int = 2, num_layers: int = 2, rngs: nnx.Rngs | None = None):
        self.in_proj = nnx.Linear(dim_in, d_model, rngs=rngs)
        self.out_proj = nnx.Linear(d_model, num_actions, rngs=rngs)
        self.layers = [Layer(d_model, rngs) for _layer_idx in range(num_layers)]

    def __call__(self, x):
        z = self.in_proj(x)
        for layer in self.layers:
            z = layer(z)
        z = self.out_proj(z)
        return z




In [ ]:
""" Policy gradients

J(\theta) = \mathbb{E}_{\tau~\pi}[reward(\tau)]

after taking gradient, we get this interesting form:
\grad_{\theta} J(\theta) = \mathbb{E}_{\tau~\pi}[Q(s, a) \grad ln(\pi(a | s))]
"""

from dataclasses import dataclass


@struct.dataclass
class Hyperparams:
    max_num_training_episodes: int
    max_num_updates: int

    seed: int = 1337
    explore_start = 1.0            # exploration probability at start
    explore_stop = 0.01            # minimum exploration probability
    decay_rate = 0.0001            # exponential decay rate for exploration prob


@dataclass
class State:
    update_idx: int
    episode_idx: int
    step_idx: int


params = Hyperparams(
    max_num_training_episodes=1000,
    max_num_updates=1000,
)
rngs = nnx.Rngs(default=params.seed)
num_actions = env.action_space.n
dim_in = env.observation_space.shape[0]

policy_model = PolicyNetwork(dim_in, params.d_model, num_actions, params.n_layers, rngs)
optimizer = nnx.Optimizer(qnet, optax.adamw(params.lr), wrt=nnx.Param)
ckpt_dir = ocp.test_utils.erase_and_create_empty(params.ckpt_dir)
checkpointer = ocp.StandardCheckpointer()

state = State(0, 0, 0)


def done_fn(state: State, params: Hyperparams):
    return (state.update_idx >= params.max_num_updates) or (state.episode_idx >= params.max_num_training_episodes)

def make_batch(trajectory) ->
while True:
    if done_fn(state, params):
        break
    current_state, _ = env.reset()
    explore_p = params.explore_stop + (params.explore_start - params.explore_stop) * np.exp(-params.decay_rate * state.update_idx)
    trajectory = play(policy_model, explore_p, env, current_state, rng=rngs, greedy=False, render=False)
    state.step_idx += len(trajectory)
    state.episode_idx += 1

